In [1]:
import pandas as pd
from PIL import Image
from scipy import misc
import numpy as np
import matplotlib.pyplot as plt
import random as rand
import cv2
%matplotlib inline

/Users/annacrawford/anaconda/envs/img_pro/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
Data=pd.read_csv('Shoe_IDs.csv')

In [3]:
Root_Location='/Users/annacrawford/Desktop/Images_CNN/'

In [4]:
files=list(Data.id)
files=[''.join(['_',str(x),'_a.jpg']) for x in files]
Image_Order=[x[1:-6] for x in files]

In [6]:
Data.head()

,Unnamed: 0,id,type_1,type_2,Model_1,Red,Green,Blue,Red_2,Green_2,Blue_2,Red_3,Green_3,Blue_3,ID
0,0,10401,sneakers,NaN,sneakers,62,58,62,190,189,191,130,129,131,10401
1,1,1557,sneakers,NaN,sneakers,51,49,57,107,106,108,29,28,33,1557
2,2,1558,boots,chelsea,boots,170,112,73,125,79,46,197,141,106,1558
3,3,1559,boots,NaN,boots,179,152,126,77,62,53,154,125,100,1559
4,4,1560,boots,chelsea,boots,25,21,21,88,86,86,46,42,42,1560


In [7]:
Background1=Image.open('Background1.jpg').convert('L')
Background2=Image.open('Background2.jpg').convert('L')
Background3=Image.open('Background3.jpg').convert('L')
Background_List=[Background1,Background2,Background3]

In [8]:
def Background_Generate(Input_Image):
    Background_Number=rand.randint(0,2)
    Background_Image=Background_List[Background_Number]
    ## Crop random area
    ar=misc.fromimage(Background_Image)
    h=ar.shape[0]
    w=ar.shape[1]
    y=rand.randint(0,h-200)
    x=rand.randint(0,w-200)
    Cropped_Image=Background_Image.crop((x,y,x+200,y+200))
    #Out_Image=Image.blend(Cropped_Image,im,0.5)
    ar_out=misc.fromimage(Cropped_Image)
    Output_Image=np.select([Input_Image<245],[Input_Image],ar_out)
    return Output_Image

In [9]:
def resize(Input_image):
    Med=cv2.resize(Input_image,(150,150))
    Small=cv2.resize(Input_image,(100,100))
    White_Small=np.empty((200,200),dtype=Med.dtype)
    White_Small.fill(255)
    White_Med=White_Small.copy()
    White_Small[50:150, 50:150] = Small
    White_Med[25:175, 25:175] = Med
    return White_Med,White_Small

In [20]:
Original_Image=[]
Background_Image=[]
Image_Rotate1=[]
Image_Rotate2=[]
Image_Rotate3=[]
Image_Scale1=[]
Image_Scale2=[]

for x in files:
    try:
        dloc=''.join([Root_Location,x])
        ## Import image in greyscale
        im = cv2.imread(dloc,0)

        ## Base Image
        Original_Image.append(im)

        ## Add Noise
        im_N=Background_Generate(im)
        Background_Image.append(im_N)

        ## Scale Image
        Med,Small=resize(im)

        ## Add Noise
        Med=Background_Generate(Med)
        Image_Scale1.append(Med)

        Small=Background_Generate(Small)
        Image_Scale2.append(Small)

        ## Noisy Image
        N_Im=Background_Generate(im)

        ## Rotate Noisy Image
        ar_90=np.rot90(N_Im,1)
        Image_Rotate1.append(ar_90)

        ar_180=np.rot90(N_Im,2)
        Image_Rotate2.append(ar_180)

        ar_270=np.rot90(N_Im,3)
        Image_Rotate3.append(ar_270)
    
    except:
        print 'Error processing image :',x

In [21]:
len(Original_Image)

9243

In [186]:
im = Image.open('/Users/annacrawford/Desktop/Images_CNN/_1557_a.jpg').convert('L')
ar = misc.fromimage(im)

In [22]:
X=np.array([Original_Image,
            Background_Image,
            Image_Rotate1,
            Image_Rotate2,
            Image_Rotate3,
            Image_Scale1,
            Image_Scale2])

In [31]:
X = np.concatenate([Original_Image,
            Background_Image,
            Image_Rotate1,
            Image_Rotate2,
            Image_Rotate3,
            Image_Scale1,
            Image_Scale2])

In [32]:
X.shape

(64701, 200, 200)

In [33]:
with file('X_Data_Large.npy', 'w') as Xoutfile:
    np.save(Xoutfile,X)

In [5]:
y=np.concatenate([Data.Model_1,
                  Data.Model_1,
                  Data.Model_1,
                  Data.Model_1,
                  Data.Model_1,
                  Data.Model_1,
                  Data.Model_1])

In [6]:
y.shape

(64701,)

In [7]:
with file('y_Data_Large.npy', 'w') as youtfile:    
    np.save(youtfile,y)

In [8]:
np.unique(y)

array(['boots', 'formal', 'loafers', 'sneakers'], dtype=object)